In [5]:
times = []

In [1]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import numpy as np
import pandas as pd

# 倒入设置参数和加载数据集函数
from Config import *


# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))
    ssCnt = {}
    for tid in D:  # 对于每一条transaction
        for can in Ck:  # 对于每一个候选项集can，检查是否是transaction的一部分 # 即该候选can是否得到transaction的支持
            flag = True
            for i in can:
                if i not in tid:
                    flag = False
                    
            if flag:
                ssCnt[can] = ssCnt.get(can, 0) + 1
                
#             if can.issubset(tid):
#                 ssCnt[can] = ssCnt.get(can, 0) + 1
    numItems = float(len(D))
    # print("ssCnt is",ssCnt)
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems  # 每个项集的支持度
        if support >= minSupport:  # 将满足最小支持度的项集，加入retList
            retList.insert(0, key)
        supportData[key] = support  # 汇总支持度数据
    return retList, supportData


def aprioriGen(Lk, k):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''
    retList = set()
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            
            L1 = Lk[i]
            L2 = Lk[j]
            cnt =0
            for m in L1:
                if m in L2:
                    cnt+=1
            if cnt == k-2:
                retList.add(Lk[i] | Lk[j])
    return retList


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    C1 = createC1(
        dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]

    D = [set(var) for var in dataSet]  # 集合化数据集
    L1, suppData = scanD(D, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素
    L = [L1]  # 最初的L1中的每个项集含有一个元素，新生成的
    # print()
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t = time.time()
        Ck = aprioriGen(L[k - 2], k)
        t1 = time.time() - t
        print(f'gen coast:{time.time() - t:.8f}s')
        
        t = time.time()
        Lk, supK = scanD(D, Ck, minSupport) # 筛选最小支持度的频繁项集
        t2 = time.time() - t
        print(f'scan coast:{time.time() - t:.8f}s')
        # print("iter is ")
        # print(Ck)
        # print(Lk)
        # print()
        times.append([t1,t2])
        suppData.update(supK)  # 将新的项集的支持度数据加入原来的总支持度字典中
        L.append(Lk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L, suppData  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList




In [2]:
myDat = loadTest()  # 导入数据集
myDat.shape


/Users/lwl/Documents/GitHub/AprioriAlgorithm/Config/__init__.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ans = np.array(ans)


(5000,)

In [7]:

import time
t = time.time()

L, suppData = apriori(myDat, para['minSupport'])  # 选择频繁项集
# print(u"频繁项集L：", suppData)
# print(u"所有候选项集的支持度信息：", suppData)
print(f'花费的时间为:{time.time() - t:.8f}s')

97
iter is  2
gen coast:0.00012994s
351
scan coast:0.35174489s
iter is  3
gen coast:0.00411487s
1704
scan coast:2.03801799s
iter is  4
gen coast:0.06388283s
4379
scan coast:6.34548306s
iter is  5
gen coast:0.45095086s
7126
scan coast:13.01727390s
iter is  6
gen coast:1.87565112s
8404
scan coast:17.00141001s
iter is  7
gen coast:5.09069681s
8129
scan coast:17.99968505s
iter is  8
gen coast:9.10212612s
6916
scan coast:16.53497791s
iter is  9
gen coast:9.66433692s
5067
scan coast:12.96398401s
iter is  10
gen coast:6.66595197s
3003
scan coast:8.18701982s
iter is  11
gen coast:2.63152528s
1365
scan coast:3.94486499s
iter is  12
gen coast:0.61496687s
455
scan coast:1.37977004s
iter is  13
gen coast:0.07631898s
105
scan coast:0.33721018s
iter is  14
gen coast:0.00485682s
15
scan coast:0.05200887s
iter is  15
gen coast:0.00017309s
1
scan coast:0.00435781s
iter is  16
gen coast:0.00000310s
0
scan coast:0.00027990s
花费的时间为:136.59765697s


In [6]:
rules = generateRules(L, suppData, minConf=0.99)
# print('rules:\n', rules)

In [8]:
times

[[0.00012993812561035156, 0.3517439365386963],
 [0.004114866256713867, 2.0380170345306396],
 [0.06388282775878906, 6.345483064651489],
 [0.45094990730285645, 13.01727294921875],
 [1.8756511211395264, 17.00140905380249],
 [5.090695858001709, 17.99968409538269],
 [9.10212516784668, 16.534976959228516],
 [9.664334774017334, 12.96398401260376],
 [6.66595196723938, 8.18701982498169],
 [2.631524085998535, 3.9448649883270264],
 [0.6149661540985107, 1.3797669410705566],
 [0.07631897926330566, 0.33720922470092773],
 [0.00485682487487793, 0.052008867263793945],
 [0.00017309188842773438, 0.004356861114501953],
 [3.0994415283203125e-06, 0.0002799034118652344]]

In [4]:
for i in L:
    print(len(i))

27
191
713
1785
3404
5230
6504
6444
5005
3003
1365
455
105
15
1
0


## CBE Apriori二进制编码算法

In [2]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import numpy as np
import pandas as pd

times = []

class node:
    def __int__(self):
        self.items = []
        self.key = 0
        self.sup = 0
    def show(self):
        print(self.items, bin(self.key), end=" ")
        try:
            print(self.sup)
        except:
            print()
            pass


def ListToNode(items):
    nodeT = node()
    nodeT.key =0
    nodeT.items = items
    for index,i in enumerate(defaultSL): # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index -1)
    return nodeT


def keyToList(key):
    ans =[]
    global defaultSL
    for i in range(1,len(defaultSL) +1):
        if key  & (1 << (len(defaultSL)) - i) == (1 << (len(defaultSL)) - i): # 含有第i个
            ans.append(defaultSL[i-1])
    return ans


# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))

    for i in Ck:
        i.sup =0
        for item in D:
            if i.key & item.key == i.key: #与运算判断子集
                i.sup +=1
    ans =[]
    length = len(D)
    for i in Ck:
        i.sup = i.sup / length
        if i.sup >= minSupport:
            ans.append(i)
            # i.show()
    return ans



def aprioriGen(Lk, k):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''

    global twoDif

    retList = []
    lenLk = len(Lk)
    nowSet = set()
    for i in range(lenLk):
        for j in range(i + 1, lenLk):

            a = Lk[i]
            b = Lk[j]
            t = a.key ^ b.key
            #要注意候选集以及去重的问题
            if t in twoDif:

                
                tkey = a.key | b.key
                if tkey not in nowSet:
                    nodeT = node()
                    nodeT.key = tkey
                    nowSet.add(nodeT.key)
                    # a.show()
                    # b.show()
#                     nodeT.items = keyToList(nodeT.key)  #生成新的items
                    retList.append(nodeT)
                else:
                    pass

    return retList


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    # C1 = createC1(dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    global defaultSL
    global defaultS

#     print(defaultSL)
    C1 = [ListToNode([i]) for i in defaultSL]


    # D = [set(var) for var in dataSet]  # 集合化数据集

    F1 = scanD(dataSet, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素

    # print()
    L = [F1]
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t= time.time()
        Ck = aprioriGen(L[k - 2], k) # 计算候选集
        t1 = time.time() - t
        print(f'gen coast:{time.time() - t:.8f}s')
        t = time.time()
        
        Fk= scanD(dataSet, Ck, minSupport) # 筛选最小支持度的频繁项集
        t2 = time.time() - t
        print(f'scan coast:{time.time() - t:.8f}s')
        
        times.append([t1,t2])
        L.append(Fk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList





In [3]:
defaultS = set()  # 存储所有单项的集合
myDat = loadTest()  # 导入数据集
for i in myDat:
    for j in i:
        if  j not in defaultS :
            defaultS.add(str(j))
defaultSL = list(defaultS)
defaultSL = sorted(defaultSL) # 把所有单项计算出来并排序，形成默认顺序，方便后面进行二进制编码
print(defaultSL, len(defaultSL))
Items = []



#对项集进行二进制编码
for items in myDat:
    nodeT = node()
    nodeT.key =0
    nodeT.items = items
    for index,i in enumerate(defaultSL): # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index -1)
    Items.append(nodeT)
    # print(items, bin(nodeT.key))


['m0e', 'm0p', 'm10e', 'm10t', 'm11b', 'm11c', 'm11e', 'm11r', 'm12f', 'm12s', 'm13f', 'm13s', 'm13y', 'm14w', 'm15w', 'm16p', 'm17w', 'm18o', 'm19e', 'm19p', 'm1b', 'm1f', 'm1s', 'm1x', 'm20k', 'm20n', 'm20u', 'm21a', 'm21n', 'm21s', 'm21v', 'm21y', 'm22d', 'm22g', 'm22m', 'm22p', 'm22u', 'm2f', 'm2s', 'm2y', 'm3g', 'm3n', 'm3w', 'm3y', 'm4f', 'm4t', 'm5a', 'm5l', 'm5n', 'm5p', 'm6f', 'm7c', 'm7w', 'm8b', 'm8n', 'm9g', 'm9k', 'm9n', 'm9p', 'm9w'] 60


得到默认编码序列['MB', 'MG', 'RB', 'RG', 'WB', 'WG', 'preparation']

In [4]:

twoDif =set() # 计算仅仅两个不同时的二进制集合
length = len(defaultSL)
for i in range(length):
    for j in range(length):
        if i != j:
            t=0
#             print(i,j)
            t = 1<<(i) 
            t+=1<<(j)
#             print(bin(t))
            twoDif.add(t)
for i in twoDif:
    print(bin(i))

0b1000000000000000100000000000000
0b10000000000001
0b100000000000001
0b11
0b1000000000000001
0b101
0b10000000000000001
0b10000000000000000001
0b1000000000000000001
0b1001
0b10000000000000000000000000000001
0b100000000000000000000000000000001
0b1010
0b1000000000000000010
0b1100
0b1000000000000000000000000000000001
0b10000000000001000
0b10001
0b10010
0b1000000000000000000010
0b10100
0b1000000000000000100
0b10000000000000000100
0b100000000000000000100
0b10000000000000000000000000000100
0b100000000000000000000000000000100
0b1000000000000000000000000000000100
0b10000000000000000000000000000000100
0b100000000000000000000000000000000100
0b1000000000000000000000000000000000100
0b10000000000000000000000000000000000100
0b10000000000000000000000000000000001
0b100000000000000000000000000000000001
0b1000000000000000000000000000000000001
0b10000000000000000000000000001
0b100000000000000000000000000001
0b1000000000000000000000000000001
0b100001
0b100010
0b100100
0b101000
0b1000000000000001000
0b10000

In [5]:
len(Items)

100

In [6]:
import time
t = time.time()

L = apriori(Items, para['minSupport'])  # 选择频繁项集
# print(u"频繁项集L：")
# for li in L:
#     for i in li:
#         i.show()
print(f'total coast:{time.time() - t:.8f}s')



60
iter is  2
gen coast:0.00229812s
528
scan coast:0.01581407s
iter is  3
gen coast:0.02532601s
4204
scan coast:0.05919600s
iter is  4
gen coast:0.40624213s
20145
scan coast:0.23848605s
iter is  5
gen coast:5.56312490s
65366
scan coast:0.75928426s
iter is  6
gen coast:41.64237309s
154034
scan coast:1.77469897s
iter is  7
gen coast:183.27786374s
275088
scan coast:3.19445205s
iter is  8
gen coast:479.85925889s
382066
scan coast:4.41344094s
iter is  9
gen coast:724.91196966s
418753
scan coast:4.85103703s
iter is  10
gen coast:680.47667503s
364446
scan coast:4.20686078s
iter is  11
gen coast:373.50062799s
251600
scan coast:2.86616111s
iter is  12
gen coast:110.55793905s
136591
scan coast:1.54438972s
iter is  13
gen coast:18.42326784s
57252
scan coast:0.64516306s
iter is  14
gen coast:1.75366902s
17928
scan coast:0.20228672s
iter is  15
gen coast:0.09379673s
3956
scan coast:0.04541922s
iter is  16
gen coast:0.00238729s
549
scan coast:0.00612378s
iter is  17
gen coast:0.00007582s
36
scan coa

### 这里对比原生的Apriori算法，可以看出答案是一样的。正确

### 关于运行速度对比
- 如果采用原生的教学数据集看不出太大差别，因为数据集很小。
- 但是如果采用稍微正常一点的不是很稠密的数据集，不难看出。随着最小支持度的减小，二者的差别越来越大
- 这里的数据集采用mushroom

### 关于运行效率分析
这里的数据集特征，
- 总计8123条数据,23个特征
- 如果数据条数足够大的时候，时间主要集中在gen上，这时候二进制编码的效率就上来了
- 当特征太少的时候，二者跑不出太大区别，因为基本都是单项集scan和查找候选集都没有太大差别

### 结果

当数据集为500,23特征，支持度为0.3
- 经典Apriori：21.2s
- 二进制Apriori：5.8s

3000,23特征，支持度为0.3
- 经典Apriori：分钟了
- 二进制Apriori：30.2s




In [7]:
for i in L:
    print(len(i))

33
362
2126
8040
21480
42698
65076
77333
72195
52935
30234
13209
4272
965
136
9
0


In [8]:
times

[[0.0022962093353271484, 0.01581287384033203],
 [0.02532505989074707, 0.05919480323791504],
 [0.40624117851257324, 0.23848509788513184],
 [5.563123941421509, 0.7592833042144775],
 [41.642372131347656, 1.7746977806091309],
 [183.2778618335724, 3.1944501399993896],
 [479.85925793647766, 4.4134392738342285],
 [724.9119679927826, 4.851036071777344],
 [680.4766731262207, 4.20685887336731],
 [373.5006239414215, 2.8661601543426514],
 [110.55793714523315, 1.544388771057129],
 [18.423266887664795, 0.6451621055603027],
 [1.7536680698394775, 0.2022867202758789],
 [0.0937957763671875, 0.04541802406311035],
 [0.0023860931396484375, 0.006123781204223633],
 [7.581710815429688e-05, 0.0004076957702636719]]